In [ ]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

import datetime
import os.path
import backtrader as bt
import pandas as pd
import numpy as np

1. Select an instrument (CL, ES, or GC) and a timeframe, (i.e. 2016-2018).  
2. Determine 20% of the timeframe as your Out-of-sample. 
2. (a) [OPTIONAL] Create an improvement to the existing strategy.  You will skip level04 and go to level05 if you choose this, mention this in the #promotion channel if you did this
3. Apply one tick of slippage per contract per trade, i.e. $10 for CL and $20 roundtrip
4. Target ~50 trades per parameter.  i.e. if you have two indicators and two parameters per indicator, you’d want about 200 trades in your in sample.  If you did not reach that number, extend your timeframe period
5. Optimize the variables for your selected timeframe for Net Profit
6. Apply your best performing parameters for your out-of-sample period



## Strategy

In [ ]:
# Create a Stratey
class TestStrategy(bt.Strategy):
    params = (
        ('bbperiod', 26),
        ('printlog', False),
        ('devfactor',2),
    )

    def log(self, txt, dt=None, doprint=False):
        ''' Logging function for this strategy'''
        if self.params.printlog or doprint:
            dt = dt or self.datas[0].datetime.date(0)
            print('%s, %s' % (dt.isoformat(), txt))

    def __init__(self):
        # Keep a reference to the "close" line in the data[0] dataseries
        self.dataclose = self.datas[0].close

        # To keep track of pending orders and buy price/commission
        self.order = None
        self.buyprice = None
        self.buycomm = None

        #indicators
#        self.sma = bt.indicators.SimpleMovingAverage(
#            self.datas[0], period=self.params.maperiod)
        self.bband = bt.indicators.BBands(self.datas[0], period = self.params.bbperiod)


    def notify_order(self, order):
        if order.status in [order.Submitted, order.Accepted]:
            # Buy/Sell order submitted/accepted to/by broker - Nothing to do
            return

        # Check if an order has been completed
        # Attention: broker could reject order if not enough cash
        if order.status in [order.Completed]:
            if order.isbuy():
                self.log(
                    'BUY EXECUTED, Price: %.2f, Cost: %.2f, Comm %.2f' %
                    (order.executed.price,
                     order.executed.value,
                     order.executed.comm))

                self.buyprice = order.executed.price
                self.buycomm = order.executed.comm
            else:  # Sell
                self.log('SELL EXECUTED, Price: %.2f, Cost: %.2f, Comm %.2f' %
                         (order.executed.price,
                          order.executed.value,
                          order.executed.comm))

            self.bar_executed = len(self)

        elif order.status in [order.Canceled, order.Margin, order.Rejected]:
            self.log('Order Canceled/Margin/Rejected')

        # Write down: no pending order
        self.order = None

    def notify_trade(self, trade):
        if not trade.isclosed:
            return

        self.log('OPERATION PROFIT, GROSS %.2f, NET %.2f' %
                 (trade.pnl, trade.pnlcomm))

    def next(self):
        # Simply log the closing price of the series from the reference
        self.log('Close, %.2f' % self.dataclose[0])

        # Check if an order is pending ... if yes, we cannot send a 2nd one
        if self.order:
            return

        # Check if we are in the market
        if not self.position:

            # Not yet ... we MIGHT BUY if ...
            if self.dataclose[0] < self.bband.lines.bot:

                # BUY, BUY, BUY!!! (with all possible default parameters)
                self.log('BUY CREATE, %.2f' % self.dataclose[0])

                # Keep track of the created order to avoid a 2nd order
                self.order = self.buy()

        else:

            if self.dataclose[0] > self.bband.lines.top:
                # SELL, SELL, SELL!!! (with all possible default parameters)
                self.log('SELL CREATE, %.2f' % self.dataclose[0])

                # Keep track of the created order to avoid a 2nd order
                self.order = self.sell()

    def stop(self):
        self.log('(BB Period %2d, devfactor %2f) Ending Value %.2f' %
                 (self.params.bbperiod, self.params.devfactor, self.broker.getvalue()), doprint=True)

helper class to analyze methods.

In [ ]:
class myAnalyzer(bt.Analyzer):
    
    def __init__(self):
        
        #self.bband = bt.indicators.BBands(self.datas[0], period = self.params.bbperiod)
        self.bbperiod = self.strategy.params.bbperiod
        self.devfactor = self.strategy.params.devfactor   
        
    def start(self):
        self.init_cash = self.strategy.broker.cash 
        self.num_trades = 0

    def notify_trade(self, trade):
        self.num_trades += 1
    
    def stop(self):
        self.final_cash = self.strategy.broker.cash
        self.final_val = self.strategy.broker.get_value()

    def get_analysis(self):
        
        output = {
            'params': (self.bbperiod, self.devfactor), #, self.cross_factor),
            'profit': self.final_val - self.init_cash,
            '#trades': self.num_trades
        }
        
        return output

## Strategy Plot

In [ ]:
class FixedSlippageAndCommisionScheme(bt.CommInfoBase):
    '''Use this for both slippage and commissions together'''
    params = (
        ('commission', 10), 
        ('stocklike', True),
        ('commtype', bt.CommInfoBase.COMM_FIXED),
        )

    def _getcommission(self, size, price, pseudoexec):
        return self.p.commission

In [ ]:
# Create a cerebro entity
cerebro = bt.Cerebro()

# Add a strategy
cerebro.addstrategy(TestStrategy)

# Load data
fname_symbol = 'CL' # use crude oil (CL)
folder_name = '5min'
suffix = '5min_20160103_20190405'

df = pd.read_parquet(os.path.join('C:/Users/A/DecisiveWorkflowResearch/data/processed/{}/'.format(folder_name), '{}_{}.parquet'.format(fname_symbol, suffix)))
df = (df.resample('2h', label='left', base=18).agg({'Open': 'first', 'High': 'max', 'Low': 'min', 'Close': 'last', 'Volume': 'sum'}))
df.columns = [col_name.lower() for col_name in df.columns]
df = df.dropna()

data = bt.feeds.PandasData(dataname = df['2016-01-01':'2017-03-31']) # in sample data

# Add the Data Feed to Cerebro
cerebro.adddata(data)

# Set our desired cash start
cerebro.broker.setcash(100000.0)

cerebro.addsizer(bt.sizers.FixedSize, stake=1000)

# We're trading futures, so we'll use the "commissions" to handle both slippage and commissions
slippage_and_comms = FixedSlippageAndCommisionScheme()
cerebro.broker.addcommissioninfo(slippage_and_comms)

# Alternatively, you can use:
# cerebro.broker.setcommission(commission=0.0)
# cerebro.broker.set_slippage_fixed(.00,  slip_open=True, slip_limit=True, slip_match=True, slip_out=True)


# Print out the starting conditions
print('Starting Portfolio Value: %.2f' % cerebro.broker.getvalue())

# Run over everything
cerebro.run()

# Print out the final result
print('Final Portfolio Value: %.2f' % cerebro.broker.getvalue())


## In-sample Optimization

In [ ]:
# Create a cerebro entity
cerebro = bt.Cerebro()

# Add a strategy
strats = cerebro.optstrategy(
    TestStrategy,
    bbperiod=range(15, 30),
    devfactor = np.linspace(1.5,3,4),
)   # range of values to let model optimized

cerebro.addanalyzer(bt.analyzers.SharpeRatio, _name='mysharpe')
cerebro.addanalyzer(myAnalyzer, _name='myAnalyzer')
cerebro.addanalyzer(bt.analyzers.DrawDown, _name='drawdown')

# Add the Data Feed to Cerebro
cerebro.adddata(data)

# Set our desired cash start
cerebro.broker.setcash(100000.0)

# Add a FixedSize sizer according to the stake
cerebro.addsizer(bt.sizers.FixedSize, stake=1000)

# We're trading futures, so we'll use the "commissions" to handle both slippage and commissions
slippage_and_comms = FixedSlippageAndCommisionScheme()
cerebro.broker.addcommissioninfo(slippage_and_comms)

# Run over everything
results = cerebro.run(maxcpus=1)

In [ ]:
params  = []
num_trades = []
profit   = []
for res in results:
    r = res[0].analyzers.myAnalyzer.get_analysis()
    params.append(r['params'])
    num_trades.append(r['#trades'])
    profit.append(r['profit'])
    
prof_ind = np.argmax(profit)
best_params = params[prof_ind]
print("Best Parameters: ",best_params,"\nTotal Profit: ", profit[prof_ind], "\nNumber of Trades: ", num_trades[prof_ind])

In [ ]:

# Create a cerebro entity
cerebro = bt.Cerebro()

# Add a strategy
cerebro.addstrategy(TestStrategy, bbperiod=19, devfactor=1.5)
cerebro.addanalyzer(myAnalyzer, _name='myAnalyzer')
cerebro.addanalyzer(bt.analyzers.SharpeRatio, _name='sharpe')
cerebro.addanalyzer(bt.analyzers.DrawDown, _name='drawdown')


data = bt.feeds.PandasData(dataname = df['2016-01-01':'2017-03-31'])

# Add the Data Feed to Cerebro
cerebro.adddata(data)

# Set our desired cash start
cerebro.broker.setcash(100000.0)
cerebro.addsizer(bt.sizers.FixedSize, stake=1000)

# We're trading futures, so we'll use the "commissions" to handle both slippage and commissions
slippage_and_comms = FixedSlippageAndCommisionScheme()
cerebro.broker.addcommissioninfo(slippage_and_comms)

# Print out the starting conditions
print('Starting Portfolio Value: %.2f' % cerebro.broker.getvalue())

# Run over everything
result_in = cerebro.run()

in_sharpe=result_in[0].analyzers.sharpe.get_analysis()
print('Sharpe Ratio:', in_sharpe)

# Print out the final result
print('Final in sample Portfolio Value: %.2f' % cerebro.broker.getvalue())

cerebro.plot(volume=False, iplot=False)

## Out-of-Sample

In [ ]:
# Create a cerebro entity
cerebro = bt.Cerebro()

# Add a strategy
cerebro.addstrategy(TestStrategy, bbperiod=19, devfactor=1.5)
cerebro.addanalyzer(bt.analyzers.SharpeRatio, _name='sharpe', timeframe=bt.TimeFrame.Months, compression=1)
cerebro.addanalyzer(bt.analyzers.DrawDown, _name='drawdown')


data2 = bt.feeds.PandasData(dataname = df['2018-01-02':'2018-07-02']) # this is out of sample


# Add the Data Feed to Cerebro
cerebro.adddata(data2)

# Set our desired cash start
cerebro.broker.setcash(100000.0)
cerebro.addsizer(bt.sizers.FixedSize, stake=1000)

# We're trading futures, so we'll use the "commissions" to handle both slippage and commissions
slippage_and_comms = FixedSlippageAndCommisionScheme()
cerebro.broker.addcommissioninfo(slippage_and_comms)

# Print out the starting conditions
print('Starting Portfolio Value: %.2f' % cerebro.broker.getvalue())

# Run over everything
result_out = cerebro.run()

cerebro.plot()
out_sharpe=result_out[0].analyzers.sharpe.get_analysis()
print('Sharpe Ratio:', out_sharpe)

# Print out the final result
print('Final out sample Portfolio Value: %.2f' % cerebro.broker.getvalue())


In [ ]:
print("In sample sharpe{}".format(in_sharpe))
print("Out of sample sharpe{}".format(out_sharpe))